<a href="https://colab.research.google.com/github/world970511/elice_OTT/blob/master/ml_ex/ott_reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**OTT 서비스 (추천)**

In [1]:
import pandas as pd
import numpy as np
import re
import urllib.request

In [2]:
!pip install plotly

##**데이터 전처리**

In [3]:
#데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/world970511/elice_OTT/master/dataset/kinolight_all_data.csv",
                        filename="OTT.csv")
data = pd.read_csv('OTT.csv',encoding='utf-8')
data.info

<bound method DataFrame.info of                                     title  ...                      provider
0                  고요의 바다(The Silent Sea)  ...                      ['넷플릭스']
1                    돈 룩 업(Don't Look Up)  ...                      ['넷플릭스']
2                           설강화(Snowdrop)  ...                      ['디즈니+']
3                     나 홀로 집에(Home Alone)  ...                      ['디즈니+']
4                        매트릭스(The Matrix)  ...  ['웨이브', '티빙', '왓챠', '쿠팡플레이']
...                                   ...  ...                           ...
27311  미니의 리본 가게 시즌 3(Minnie's Bow-Toons)  ...                      ['디즈니+']
27312         만능 수리공 매니 시즌 1(Handy Manny)  ...                      ['디즈니+']
27313         만능 수리공 매니 시즌 2(Handy Manny)  ...                      ['디즈니+']
27314          아기를 부탁해! 토츠 시즌 2(T.O.T.S.)  ...                      ['디즈니+']
27315            실험실의 비밀친구 시즌 3(Lab Rats)  ...                      ['디즈니+']

[27316 rows x 10 columns]>

In [4]:
#중복제거 및 기타 제거
li=['genre','country','cast','director','provider',]

for i in li:
  data[i]=data[i].apply(lambda x:re.sub(r'\([^)]*\)', '',x.replace('[','').replace(']','').replace("'",'').replace("'",'')))
  data[i]=data[i].apply(lambda x:re.sub(r' ', '',x))
  data[i]=data[i].apply(lambda x: ' '.join(set(x.strip().split(','))))

data.head(5)

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021.0,TV,15세이상관람가,SF 스릴러,한국,배두나 공유 이준 이성욱 허성태 이무생 김선영,최항용,NaN,넷플릭스
1,돈 룩 업(Don't Look Up),2021.0,movie,15세이상관람가,코미디,미국,롭모건 마크라이런스 레오나르도디카프리오 제니퍼로렌스 조나힐 타일러페리 티모시샬라메,아담맥케이,139.0,넷플릭스
2,설강화(Snowdrop),2021.0,TV,15세이상관람가,드라마,한국,지수 윤세아 정유진 유인나 장승조 김혜윤 정해인,조현탁,NaN,디즈니+
3,나 홀로 집에(Home Alone),1990.0,movie,전체관람가,코미디 가족 어드벤처,미국,조페시 안젤라고덜즈 다니엘스턴 맥컬리컬킨 존허드 로버츠블로썸 캐서린오하라,크리스콜럼버스,105.0,디즈니+
4,매트릭스(The Matrix),1999.0,movie,12세이상관람가,SF 액션,미국,캐리앤모스 글로리아포스터 조판토리아노 휴고위빙 키아누리브스 로렌스피시번,릴리워쇼스키 라나워쇼스키,136.0,쿠팡플레이 왓챠 티빙 웨이브


In [5]:
#불필요한 행 제거
data= data[data['title']!='title']

#년도 정수형으로 변경
data['year']=data['year'].astype('float').astype('Int64')

#제작미정,시사/교양,기타(장르가 아니라 country로 가있음;;) 정리
# 제작미정인 경우 풀릴 예정인 곳과 아닌 곳이 있고 양이 너무 많아서 그냥 장르로 변경함
def r_ch(s,i):
  li=s.split(',')
  if i in li: 
    li.remove(i)
  return ','.join (li)


def add_ch(s,i):
  li=s.split(' ')
  if i not in li: li+=[i]
  return ' '.join (li)

li=['미정','시사/교양','기타']
for i in li:
    d=data.loc[data["country"].str.contains(i)]
    data=data.loc[data["country"].str.contains(i)==False]

    d['country']=d['country'].apply(lambda x:r_ch(x,i))
    d['genre']=d['genre'].apply(lambda x:add_ch(x,i))

    data=pd.concat([data,d])

data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021,TV,15세이상관람가,SF 스릴러,한국,배두나 공유 이준 이성욱 허성태 이무생 김선영,최항용,NaN,넷플릭스
1,돈 룩 업(Don't Look Up),2021,movie,15세이상관람가,코미디,미국,롭모건 마크라이런스 레오나르도디카프리오 제니퍼로렌스 조나힐 타일러페리 티모시샬라메,아담맥케이,139.0,넷플릭스
2,설강화(Snowdrop),2021,TV,15세이상관람가,드라마,한국,지수 윤세아 정유진 유인나 장승조 김혜윤 정해인,조현탁,NaN,디즈니+
3,나 홀로 집에(Home Alone),1990,movie,전체관람가,코미디 가족 어드벤처,미국,조페시 안젤라고덜즈 다니엘스턴 맥컬리컬킨 존허드 로버츠블로썸 캐서린오하라,크리스콜럼버스,105.0,디즈니+
4,매트릭스(The Matrix),1999,movie,12세이상관람가,SF 액션,미국,캐리앤모스 글로리아포스터 조판토리아노 휴고위빙 키아누리브스 로렌스피시번,릴리워쇼스키 라나워쇼스키,136.0,쿠팡플레이 왓챠 티빙 웨이브
...,...,...,...,...,...,...,...,...,...,...
24575,셜록홈즈의 악마의 조력자들(Sherlock Holmes - Hands of a Mu...,1990,movie,NaN,어드벤처 액션 미스터리 기타,기타 영국,에드워드우드워드 존힐러맨,스튜어트옴,NaN,왓챠 티빙
25022,JOB룡 이십끼(),2020,TV,NaN,예능 기타,기타 한국,유민상,,NaN,왓챠 웨이브
26598,빌로우 덱: 지중해의 요티 시즌 2(Below Deck Mediterranean),2017,TV,NaN,예능 기타,미국 기타,,,NaN,넷플릭스
23607,쉘위워크(Shall We Walk?),2017,TV,NaN,예능 시사/교양 기타,시사/교양 기타 한국,이제훈 이하늬,,NaN,티빙


In [6]:
#독점 및 오리지널 표시 및 필요한 provider데이터 외에 드랍

##필요한 provider외에 나머지 드랍
remove_provider_list=['네이버시리즈온', 'GooglePlay무비', '씨네폭스','아마존프라임비디오','AppleTV+']
n_data=pd.DataFrame(columns=['title',	'year',	'kind',	'KMRB',	'genre',	'country',	'cast',	'director',	'runtime(min)',	'provider'])
for i in remove_provider_list:
  data['provider']=data['provider'].apply(lambda x: x.replace(i,''))

data['provider'].dropna()

#provider 리스트 전환
data['provider']=data['provider'].apply(lambda x: x.split(','))

#독점 여부
data.loc[:,'origin']=data['provider'].apply(lambda x: True if len(x)==1 else False)

#provider 다시 문자열 전환
data['provider']=data['provider'].apply(lambda x: ','.join(x))

data.to_csv('/content/drive/MyDrive/project/엘리스 OTT 분석/data/all_DATA_ processing.csv',mode='w', index=False)
data.head(5)

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider,origin
0,고요의 바다(The Silent Sea),2021,TV,15세이상관람가,SF 스릴러,한국,배두나 공유 이준 이성욱 허성태 이무생 김선영,최항용,NaN,넷플릭스,True
1,돈 룩 업(Don't Look Up),2021,movie,15세이상관람가,코미디,미국,롭모건 마크라이런스 레오나르도디카프리오 제니퍼로렌스 조나힐 타일러페리 티모시샬라메,아담맥케이,139.0,넷플릭스,True
2,설강화(Snowdrop),2021,TV,15세이상관람가,드라마,한국,지수 윤세아 정유진 유인나 장승조 김혜윤 정해인,조현탁,NaN,디즈니+,True
3,나 홀로 집에(Home Alone),1990,movie,전체관람가,코미디 가족 어드벤처,미국,조페시 안젤라고덜즈 다니엘스턴 맥컬리컬킨 존허드 로버츠블로썸 캐서린오하라,크리스콜럼버스,105.0,디즈니+,True
4,매트릭스(The Matrix),1999,movie,12세이상관람가,SF 액션,미국,캐리앤모스 글로리아포스터 조판토리아노 휴고위빙 키아누리브스 로렌스피시번,릴리워쇼스키 라나워쇼스키,136.0,쿠팡플레이 왓챠 티빙 웨이브,True


##**추천**

In [7]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

####TF-IDF 벡터화 
전처리한 데이터를 TF-IDF 방법을 이용해 벡터로 변경

장르/국가/종류를 하나로 합친 후 tfidf vector화.

https://github.com/lsjsj92/recommender_system_with_Python/blob/master/002.%20recommender%20system%20basic%20with%20Python%20-%201%20content%20based%20filtering.ipynb 참고

In [8]:
tfidf_vector = TfidfVectorizer()
tfidf_matrix = tfidf_vector.fit_transform(+data['genre'] + " " + data['country']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names()
tfidf_matrix.shape

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(27301, 149)

In [9]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = data.title)
print(tfidf_matrix.shape)
tfidf_matrix.head()

(27301, 149)


,sf,가나,가족,공연,공포,과테말라,교양,그리스,기타,나이지리아,남아프리카공화국,네덜란드,네팔,노르웨이,뉴질랜드,다큐멘터리,대만,덴마크,도미니카,독일,드라마,라오스,라트비아,러시아,레바논,로맨스,루마니아,룩셈부르크,리비아,리투아니아,마케도니아,말레이시아,말리,멕시코,멜로,모로코,모리셔스,모리타니,몰타,몽골,...,일본,전쟁,제한상영가,조지아,중국,체코,칠레,카메룬,카자흐스탄,카타르,캄보디아,캐나다,케냐,코미디,콜롬비아,콩고민주공화국,쿠바,크로아티아,키르기스스탄,키즈,탄자니아,태국,터키,튀니지,파나마,파라과이,파키스탄,판타지,팔레스타인,페루,포르투갈,폴란드,푸에르토리코,프랑스,핀란드,필리핀,한국,헝가리,호주,홍콩
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
고요의 바다(The Silent Sea),0.725652,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.403121,0.0,0.0,0.0
돈 룩 업(Don't Look Up),0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
설강화(Snowdrop),0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.632556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.774514,0.0,0.0,0.0
나 홀로 집에(Home Alone),0.000000,0.0,0.644745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.414717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
매트릭스(The Matrix),0.757480,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


####유사도
tf-idf vector를 코사인 유사도를 활용해서 유사도 값을 구하기


In [10]:
%%time
cosine_sim = cosine_similarity(tfidf_matrix)

CPU times: user 10.9 s, sys: 2.25 s, total: 13.2 s
Wall time: 11.7 s


In [11]:
cosine_sim.shape

(27301, 27301)

In [12]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = data.title, columns = data.title)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(27301, 27301)


title,고요의 바다(The Silent Sea),돈 룩 업(Don't Look Up),설강화(Snowdrop),나 홀로 집에(Home Alone),매트릭스(The Matrix),기적(Miracle),엔칸토: 마법의 세계(Encanto),그 해 우리는(Our Beloved Summer),스파이더맨(Spider-Man),불가살(Bulgasal: Immortal Souls),모가디슈(Escape from Mogadishu),스파이더맨: 파 프롬 홈(Spider-Man: Far From Home),옷소매 붉은 끝동(The Red Sleeve),인셉션(Inception),크루엘라(Cruella),귀멸의 칼날: 환락의 거리편(Demon Slayer: Kimetsu no Yaiba Yuukaku-hen),킹스맨: 골든 서클(Kingsman: The Golden Circle),킹스맨 : 시크릿 에이전트(Kingsman: The Secret Service),캐롤(Carol),러브 레터(Love Letter),배드 앤 크레이지(Bad and Crazy),해리 포터와 마법사의 돌(Harry Potter And The Sorcerer's Stone),스파이더맨: 홈 커밍(Spider-Man: Homecoming),어메이징 스파이더맨(The Amazing Spider - Man),로맨틱 홀리데이(The Holiday),스파이더맨: 뉴 유니버스(Spider-Man: Into the Spider-Verse),어벤져스: 엔드게임(Avengers: Endgame),호크아이(Hawkeye),다크 나이트(The Dark Knight),프라미싱 영 우먼(Promising Young Woman),술꾼도시여자들(Work Later Drink Now),위대한 개츠비(The Great Gatsby),베놈(Venom),완다비전(WandaVision),해피니스(Happiness),가타카(Gattaca),미나리(Minari),위대한 쇼맨(The Greatest Showman),나 홀로 집에 2 - 뉴욕을 헤매다(Home Alone 2),스파이더맨 2(Spider-Man II),...,하트어택(Heart_a_tag),사라진 기억(Vanishing Waves),대화가 필요한 개냥(),인간이 왜 그래(),트랜스페어런트 시즌 5(Transparent Season 5),궁민남편(),알제리 전투(The Battle Of Algiers),집을 파는 여자(Your Home is My Business),꽃미남 라면가게(),해파리 공주(),여명의 눈동자(),이 죽일놈의 사랑(),피리부는 사나이(Pied Piper),천년째 연애중(Love For A Thousand More),힙한 선생(hiphopteacher),북극의 후예 이누크(Inuk),더 바이러스(The Virus),슈퍼스타 K 2016(Superstar K 2016),예능인력소(),짝(),MBC 베스트극장(),사이비 육식녀의 연애사정(),"그남자 오수(That Man, Oh Soo)",조아서 구독중 시즌 2(),내 여자친구가 너무 성실한 처녀 빗치인 건(MY GIRLFRIEND IS SHOBITCH),젠틀맨리그(GentleMan League),열정 같은 소리(),연애 드라마 같은 사랑을 하고싶어(Kiss On The Bed),마약 전쟁(Talento de barrio),라이브 테러(A Common Man),저스트 비트윈 어스(Just Between Us),아담의 팩트폭격 시즌 1(Adam Ruins Everything),주먹이 운다 시즌 1(),외계 통신(),삼총사 시즌 3(The Musketeers Season 3),셜록홈즈의 악마의 조력자들(Sherlock Holmes - Hands of a Murderer),JOB룡 이십끼(),빌로우 덱: 지중해의 요티 시즌 2(Below Deck Mediterranean),쉘위워크(Shall We Walk?),TV정보쇼 오!아시스()
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
고요의 바다(The Silent Sea),1.000000,0.000000,0.312223,0.000000,0.549667,0.312223,0.000000,0.178357,0.290526,0.312223,0.223805,0.416015,0.312223,0.678411,0.000000,0.0,0.000000,0.292675,0.000000,0.000000,0.312223,0.000000,0.451368,0.000000,0.000000,0.000000,0.549667,0.000000,0.000000,0.264655,0.312223,0.000000,0.651499,0.000000,0.312223,0.600165,0.000000,0.000000,0.251691,0.647480,...,0.060810,0.267976,0.060810,0.060810,0.000000,0.060810,0.000000,0.000000,0.058307,0.000000,0.062050,0.062050,0.059713,0.057757,0.060987,0.000000,0.171766,0.060810,0.060810,0.062050,0.062050,0.000000,0.059665,0.062050,0.0,0.060810,0.060810,0.0,0.000000,0.115155,0.000000,0.000000,0.060810,0.060810,0.000000,0.000000,0.060810,0.000000,0.040644,0.040644
돈 룩 업(Don't Look Up),0.000000,1.000000,0.000000,0.517836,0.229531,0.000000,0.350195,0.000000,0.205462,0.000000,0.000000,0.484423,0.000000,0.178119,1.000000,0.0,0.500300,0.206983,0.159795,0.000000,0.000000,0.158875,0.188483,0.240718,0.600209,0.197728,0.229531,0.351579,0.183591,0.187166,0.000000,0.160219,0.171053,0.226458,0.000000,0.250618,0.445860,0.191244,0.496352,0.270376,...,0.000000,0.000000,0.000000,0.000000,0.230563,0.000000,0.000000,0.000000,0.141106,0.139853,0.000000,0.000000,0.000000,0.000000,0.147591,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.140822,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.081439,0.128428,0.226090,0.000000,0.000000,0.000000,0.000000,0.000000,0.082442,0.000000,0.000000
설강화(Snowdrop),0.312223,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.342677,0.000000,1.000000,0.716810,0.000000,1.000000,0.168700,0.000000,0.0,0.000000,0.000000,0.151345,0.200551,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.173883,0.177269,1.000000,0.151747,0.000000,0.214483,1.000000,0.237366,0.422284,0.181131,0.000000,0.000000,...,0.116834,0.000000,0.116834,0.116834,0.078311,0.116834,0.071875,0.078722,0.186749,0.074060,0.198736,0.198736,0.114726,0.110968,0.195332,0.075072,0.113279,0.1168

####CBF
선택한 영화 제목에 따라서 코사인 유사도를 구한 matrix에서 유사도 데이터를 바탕으로 가장 유사도 값이 큰 데이터를 가져옴


In [16]:
def recommendations(target_title, matrix, items):
    recom_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:100+1]
    recom_title = items.iloc[recom_idx, :].title.values
    recom_kind = items.iloc[recom_idx, :].kind.values
    recom_genre = items.iloc[recom_idx, :].genre.values
    recom_country = items.iloc[recom_idx, :].country.values
    recom_provider = items.iloc[recom_idx, :].provider.values

    target_title_list = np.full(len(range(100)), target_title)
    target_genre_list = np.full(len(range(100)), items[items.title == target_title].genre.values)
    
    d = {
        'recom_title' : recom_title,
        'recom_kind':recom_kind,
        'recom_genre' : recom_genre,
        'recom_country':recom_country,
        'recom_provider':recom_provider
    }
    return pd.DataFrame(d)

In [19]:
data.loc[data['title']=='스파이더맨: 파 프롬 홈(Spider-Man: Far From Home)']

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider,origin
11,스파이더맨: 파 프롬 홈(Spider-Man: Far From Home),2019,movie,12세이상관람가,SF 코미디 액션 어드벤처,미국,존파브로 톰홀랜드 젠데이아콜먼 제이크질렌할 사무엘L.잭슨 마리사토메이 코비스멀더스,존왓츠,129.0,왓챠 seezn 넷플릭스,True


In [18]:
d=recommendations('스파이더맨: 파 프롬 홈(Spider-Man: Far From Home)', cosine_sim_df, data)
d.head(10)

                                       title  year  ...       provider origin
11  스파이더맨: 파 프롬 홈(Spider-Man: Far From Home)  2019  ...  왓챠 seezn 넷플릭스   True

[1 rows x 11 columns]


,recom_title,recom_kind,recom_genre,recom_country,recom_provider
0,림 오브 더 월드(Rim of the World),movie,SF 코미디 액션 어드벤처,미국,넷플릭스
1,스파이더맨: 파 프롬 홈(Spider-Man: Far From Home),movie,SF 코미디 액션 어드벤처,미국,왓챠 seezn 넷플릭스
2,혹성탈출(Planet Of The Apes),movie,액션 코미디 SF 드라마 어드벤처,미국,디즈니+
3,맥스 스틸(Max Steel),movie,SF 액션 어드벤처,미국,왓챠
4,트랜스포머: 최후의 기사(Transformers: The Last Knight),movie,SF 액션 어드벤처,미국,넷플릭스
5,타임 어게인(Time Again),movie,SF 액션 어드벤처,미국,왓챠 웨이브
6,스파이더맨: 홈 커밍(Spider-Man: Homecoming),movie,SF 액션 어드벤처,미국,왓챠 쿠팡플레이 넷플릭스
7,트랜스포머 3(Transformers: Dark Of The Moon),movie,SF 액션 어드벤처,미국,왓챠 웨이브 티빙 넷플릭스
8,헬보이(Hellboy),movie,SF 액션 어드벤처,미국,쿠팡플레이 왓챠 웨이브
9,애프터 어스(After Earth),movie,SF 액션 어드벤처,미국,넷플릭스


####추가 test-CF(Matrix Factorization)

In [44]:
#사용자 선호 데이터 - 10개에 대한 데이터를 받았다고 가정
title = ['스파이더맨: 파 프롬 홈(Spider-Man: Far From Home)','나 홀로 집에(Home Alone)',
               '매트릭스(The Matrix)','옷소매 붉은 끝동(The Red Sleeve)',
               '귀멸의 칼날: 환락의 거리편(Demon Slayer: Kimetsu no Yaiba Yuukaku-hen)',
               '러브 레터(Love Letter)','술꾼도시여자들(Work Later Drink Now)',
               '오징어 게임(Squid Game)','그녀(Her)','응답하라 1988(Reply 1988)']

li=[]
user=pd.DataFrame(columns=['user','title','year','kind','KMRB','genre','country','cast','director','runtime(min)','provider','origin','rating'])
for i in title:
  u=['나은']
  u+= data.loc[data['title']==i].values.tolist()[0]
  u.append(5.0)
  user.loc[len(user)] = u

user

,user,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider,origin,rating
0,나은,스파이더맨: 파 프롬 홈(Spider-Man: Far From Home),2019,movie,12세이상관람가,SF 코미디 액션 어드벤처,미국,존파브로 톰홀랜드 젠데이아콜먼 제이크질렌할 사무엘L.잭슨 마리사토메이 코비스멀더스,존왓츠,129.0,왓챠 seezn 넷플릭스,True,5.0
1,나은,나 홀로 집에(Home Alone),1990,movie,전체관람가,코미디 가족 어드벤처,미국,조페시 안젤라고덜즈 다니엘스턴 맥컬리컬킨 존허드 로버츠블로썸 캐서린오하라,크리스콜럼버스,105.0,디즈니+,True,5.0
2,나은,매트릭스(The Matrix),1999,movie,12세이상관람가,SF 액션,미국,캐리앤모스 글로리아포스터 조판토리아노 휴고위빙 키아누리브스 로렌스피시번,릴리워쇼스키 라나워쇼스키,136.0,쿠팡플레이 왓챠 티빙 웨이브,True,5.0
3,나은,옷소매 붉은 끝동(The Red Sleeve),2021,TV,15세이상관람가,드라마,한국,준호 강훈 이덕화 장혜진 박지영 이세영 장희진,정지인,NaN,쿠팡플레이 웨이브,True,5.0
4,나은,귀멸의 칼날: 환락의 거리편(Demon Slayer: Kimetsu no Yaiba...,2021,TV,청소년관람불가,애니메이션,일본,코니시카츠유키 키토아카리 하나에나츠키 시모노히로 마츠오카요시츠구,소토자키하루오,NaN,왓챠 티빙 라프텔 웨이브,True,5.0
5,나은,러브 레터(Love Letter),1995,movie,전체관람가,드라마 멜로/로맨스,일본,카시와바라타카시 사카이미키 스즈키란란 나카야마미호 한분샤쿠 시노하라카츠유키 토요카와에츠시,이와이슌지,116.0,왓챠,True,5.0
6,나은,술꾼도시여자들(Work Later Drink Now),2021,TV,청소년관람불가,드라마,한국,최시원 이선빈 한선화 김정민 정은지,김정식,NaN,티빙,True,5.0
7,나은,오징어 게임(Squid Game),2021,TV,청소년관람불가,드라마 스릴러,한국,위하준 정호연 박해수 이정재 오영수 허성태 김주령,황동혁,NaN,넷플릭스,True,5.0
8,나은,그녀(Her),2013,movie,15세이상관람가,드라마 멜로/로맨스,미국,루니마라 호아킨피닉스 에이미아담스 올리비아와일드 크리스프랫 스칼릿조핸슨 맷레처,스파이크존즈,126.0,왓챠 seezn 쿠팡플레이 넷플릭스,True,5.0
9,나은,응답하라 1988(Reply 1988),2015,TV,15세이상관람가,코미디 가족,한국,라미란 류준열 성동일 김성균 이일화 박보검 이혜리,신원호,NaN,티빙,True,5.0
